In [1]:
import sys
if 'google.colab' in sys.modules:
    !git clone  https://github.com/ecastillot/delaware.git ./delaware
    !pip install obspy
    

In [2]:
import sys
import os

version = "10102024"

if 'google.colab' in sys.modules:
    dw_path = os.path.join("/content/delaware",version)
else:
    dw_path = os.path.join("/home/emmanuel/ecastillo/dev/delaware",version)
    
sys.path.append(dw_path)

# Stations

In [3]:
import pandas as pd
import os

stations_relpath = "data_git/stations/standard_stations.csv"
stations_path = os.path.join(dw_path,stations_relpath)
stations = pd.read_csv(stations_path)
stations_columns = ["network","station","latitude","longitude","elevation","x[km]","y[km]"]
stations = stations[stations_columns]
stations

,network,station,latitude,longitude,elevation,x[km],y[km]
0,4O,WB01,31.721667,-104.060278,0.92700,-11583.937159,3726.830293
1,4O,CT01,31.902850,-104.144532,0.99400,-11593.316271,3750.564925
2,4O,SA02,31.671630,-104.264900,0.76000,-11606.715576,3720.283725
3,4O,SA04,31.755930,-104.254580,0.76000,-11605.566758,3731.315115
4,4O,SA06,31.756960,-104.149710,0.76000,-11593.892683,3731.449961
5,4O,WB02,31.736302,-103.944920,0.93200,-11571.095621,3728.745711
6,4O,WB03,31.610722,-103.968839,0.99400,-11573.758171,3712.319547
7,4O,SA07,31.865780,-104.364810,1.10300,-11617.837506,3745.705033
8,4O,WB04,31.530513,-103.950319,0.96189,-11571.696579,3701.839715
9,4O,WB05,31.753781,-103.832948,0.85481,-11558.630888,3731.033735


# Custom Client: Catalog with picks

In [8]:
from obspy import UTCDateTime
from delaware.core.client import CustomClient
from delaware.loc.inv import prepare_cat2inv

provider = "USGS"
sta_ref = "PB28"
radius = 1
starttime = UTCDateTime(f"2023-01-01 15:26:30")
endtime = UTCDateTime("2024-09-01 15:26:32")

client =  CustomClient(provider)

ref = stations[stations["station"].isin([sta_ref])]
ref = ref.drop_duplicates(subset=["station"],
                                          ignore_index=True)

cat,picks,mag = client.get_custom_events(
                        latitude=ref.loc[0].latitude,
                        longitude=ref.loc[0].longitude,
                        maxradius=radius/111,
                        includeallorigins=True,
                        starttime=starttime,
                        endtime=endtime)

cat, picks = prepare_cat2inv(cat,picks,attach_station=stations)
print("### catalog ###\n",cat)
print("### picks ###\n",picks)

### catalog ###
              ev_id                origin_time  eq_latitude  eq_longitude  \
0   texnet2023aprf 2023-01-09 13:44:01.270139    31.673565   -104.503905   
1   texnet2023dfli 2023-02-15 15:59:46.894342    31.664441   -104.494662   
2   texnet2023dxyu 2023-02-25 19:15:06.950018    31.666939   -104.498724   
3   texnet2023ecja 2023-02-28 04:58:17.588601    31.669947   -104.495793   
4   texnet2023eohi 2023-03-06 17:43:34.425815    31.671813   -104.492044   
5   texnet2023kxgo 2023-06-05 00:47:20.115651    31.673188   -104.492596   
6   texnet2023mvis 2023-07-02 10:41:25.602654    31.670780   -104.507645   
7   texnet2023tkhy 2023-10-04 00:33:18.295847    31.668000   -104.506000   
8   texnet2023utrt 2023-10-23 09:15:39.405804    31.665000   -104.497000   
9   texnet2023vqxf 2023-11-05 02:08:37.113575    31.674000   -104.496000   
10  texnet2023vrfn 2023-11-05 06:20:14.330848    31.668000   -104.497000   
11  texnet2023vrkq 2023-11-05 08:56:50.155112    31.672000   -104.49800

In [9]:
picks = picks[picks["station"]==sta_ref]
picks

,ev_id,station,r,az,tt_P,tt_S
0,texnet2024irrh,PB28,0.864813,115.516114,1.026446,1.714365
1,texnet2023mvis,PB28,0.693770,110.000712,1.314721,2.054923
2,texnet2023xbaq,PB28,0.807840,222.635649,1.453334,2.472544
3,texnet2023vrkq,PB28,0.455852,215.186945,1.481353,2.447761
4,texnet2024rdjx,PB28,0.708974,185.911615,1.488855,2.390264
5,texnet2023kxgo,PB28,0.924732,236.956337,1.551804,2.518008
6,texnet2023xcih,PB28,0.693282,247.825975,1.570631,2.565955
7,texnet2023eohi,PB28,0.899182,246.968555,1.592348,2.563636
9,texnet2023vqxf,PB28,0.746876,217.275640,1.597551,2.574851
10,texnet2023xgya,PB28,0.516346,223.819841,1.605712,2.663527


# Particle Swarm Optimization

In [13]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess

    # Compute the errors (residuals)
    tp_error = (tp_pred - tp_obs)**2
    ts_error = (ts_pred - ts_obs)**2
    loss = tp_error + ts_error

    return loss

def vd_pso(cost_func,
           picks,
           bounds,
           max_iter=100,w=0.5,c1=0.8,c2=0.9):
    
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    velocities = np.zeros((n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r",'tt_P','tt_S']}
    
    # Initialize the particles and velocities
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, 3))
    velocities = np.zeros((n_particles, 3))

    # Initialize the best positions and best costs
    best_positions = particles.copy()
    best_costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

    # Initialize the global best position and global best cost
    global_best_position = particles[0].copy()
    global_best_cost = best_costs[0]

    # Perform the optimization
    for i in range(max_iter):
        # Update the velocities
        r1 = np.random.rand(n_particles, 3) #Random matrix used to compute the cognitive component of the veocity update
        r2 = np.random.rand(n_particles, 3) #Random matrix used to compute the social component of the veocity update


        #Cognitive component is calculated by taking the difference between the
        #particle's current position and its best personal position found so far,
        #and then multiplying it by a random matrix r1 and a cognitive acceleration coefficient c1.
        cognitive_component = c1 * r1 * (best_positions - particles)

        #The social component represents the particle's tendency to move towards the
        #global best position found by the swarm. It is calculated by taking the
        #difference between the particle's current position and the global best position
        # found by the swarm, and then multiplying it by a random matrix r2 and a
        #social acceleration coefficient c2.
        social_component = c2 * r2 * (global_best_position - particles)

        #The new velocity of the particle is computed by adding the current velocity
        #to the sum of the cognitive and social components, multiplied by the inertia
        #weight w. The new velocity is then used to update the position of the
        #particle in the search space.
        velocities = w * velocities + cognitive_component + social_component

        # Update the particles
        particles += velocities

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the objective function
        costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

        # Update the best positions and best costs
        is_best = costs < best_costs
        best_positions[is_best] = particles[is_best]
        best_costs[is_best] = costs[is_best]

        # Update the global best position and global best cost
        global_best_index = np.argmin(best_costs)
        global_best_position = best_positions[global_best_index].copy()
        global_best_cost = best_costs[global_best_index]

        # Print the progress
        if (i+1)%10 == 0:
            print(f'Iteration {i+1}: Best Cost = {global_best_cost} | sol:{global_best_position}')
    
    return global_best_position, global_best_cost



cost_func = loss_function


bounds = np.array([[0,2,2/2],[10,4.7,4.7/2]])

# Run the PSO algorithm on the Rastrigin function
solution, fitness = vd_pso(cost_func=cost_func,picks=picks,
                            bounds=bounds,max_iter=500
                            )

print("Best Solution [z,vp,vs]:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 4.263874282337582e-05 | sol:[4.5444096  2.91558677 1.78983973]
Iteration 20: Best Cost = 1.4021118177572627e-05 | sol:[4.53551406 2.91643509 1.78999106]
Iteration 30: Best Cost = 1.313621777211737e-05 | sol:[4.53636396 2.91644722 1.78997489]
Iteration 40: Best Cost = 1.2959723263714479e-05 | sol:[4.53698786 2.91644816 1.78996998]
Iteration 50: Best Cost = 1.2959628453765611e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 60: Best Cost = 1.2959628361192149e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 70: Best Cost = 1.2959628361102428e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 80: Best Cost = 1.2959628361102428e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 90: Best Cost = 1.2959628361102428e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 100: Best Cost = 1.2959628361102428e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 110: Best Cost = 1.2959628361102428e-05 | sol:[4.53698801 2.91644817 1.78996996]
Iteration 

# Gradient Descent

In [23]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

def vd_gradient_descent(cost_func, picks, bounds, max_iter=100, lr=0.01):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize the best position and best cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform gradient descent optimization
    for i in range(max_iter):
        # Compute gradients of the loss function
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])
        
        # Update particles using gradient descent
        particles -= lr * gradients

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost} | sol:{best_position}')

    return best_position, best_cost

# Example of how to call the Gradient Descent-based optimization function
bounds = np.array([[0,2,2/2],[10,4.7,4.7/2]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_gradient_descent(cost_func=loss_function, picks=picks, bounds=bounds, max_iter=500, lr=0.01)

print("Best Solution [z,vp,vs]:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.1299568270295993 | sol:[4.18454395 3.00864235 1.9651263 ]
Iteration 20: Best Cost = 0.07008503787701412 | sol:[4.22180664 2.99727322 1.89794561]
Iteration 30: Best Cost = 0.036048451708270826 | sol:[4.24986233 2.9872756  1.84752926]
Iteration 40: Best Cost = 0.018577414743875382 | sol:[4.27037769 2.9783953  1.81203899]
Iteration 50: Best Cost = 0.009528371187913035 | sol:[3.64779687 2.3188595  1.53571002]
Iteration 60: Best Cost = 0.0033282723699234957 | sol:[3.65982532 2.31339454 1.51458584]
Iteration 70: Best Cost = 0.0013594646970876692 | sol:[3.66707536 2.30894117 1.50343087]
Iteration 80: Best Cost = 0.0007105511109522612 | sol:[3.67158583 2.305214   1.49789626]
Iteration 90: Best Cost = 0.0004568294603461242 | sol:[3.6745535  2.30203528 1.49534954]
Iteration 100: Best Cost = 0.00022379919901061585 | sol:[5.25024041 3.29929267 1.99066301]
Iteration 110: Best Cost = 0.00011082340380093805 | sol:[5.25173445 3.29867616 1.98770369]
Iteration 120: Best Cost 

# Adam

In [26]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def vd_adam(cost_func, picks, bounds, max_iter=100, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize moment estimates
    m = np.zeros_like(particles)  # First moment (mean of gradients)
    v = np.zeros_like(particles)  # Second moment (uncentered variance of gradients)
    t = 0  # Time step (for bias correction)

    # Evaluate initial cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform Adam optimization
    for i in range(max_iter):
        t += 1  # Increment time step

        # Compute gradients (numerical approximation)
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])

        # Update moment estimates (bias correction applied to m and v)
        m = beta1 * m + (1 - beta1) * gradients  # Update first moment
        v = beta2 * v + (1 - beta2) * gradients**2  # Update second moment

        # Bias correction (to counteract initialization bias)
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)

        # Update particles using Adam's update rule
        particles -= lr * m_hat / (np.sqrt(v_hat) + epsilon)

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost} | sol:{best_position}')

    return best_position, best_cost

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

# Example of how to call the Adam-based optimization function
bounds = np.array([[0,2,2/2],[10,4,4/2]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_adam(cost_func=loss_function, picks=picks, bounds=bounds, max_iter=500, lr=0.001)

print("Best Solution: [z,vp,vs]", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.21850262405144621 | sol:[3.98711262 3.56343396 1.9439945 ]
Iteration 20: Best Cost = 0.20492813964026227 | sol:[3.99704324 3.55345198 1.93407369]
Iteration 30: Best Cost = 0.19194022132820587 | sol:[4.0068663  3.54349772 1.9242786 ]
Iteration 40: Best Cost = 0.1795837582636815 | sol:[4.01654505 3.53358039 1.91465636]
Iteration 50: Best Cost = 0.16788317361211139 | sol:[4.02605444 3.52370611 1.90524253]
Iteration 60: Best Cost = 0.15684808112818843 | sol:[4.03537778 3.51387885 1.8960643 ]
Iteration 70: Best Cost = 0.1464783075286458 | sol:[4.0445034  3.5041013  1.88714388]
Iteration 80: Best Cost = 0.1367670976036384 | sol:[4.05342247 3.49437549 1.87850093]
Iteration 90: Best Cost = 0.1277028224833059 | sol:[4.06212769 3.4847031  1.87015382]
Iteration 100: Best Cost = 0.11926981086757737 | sol:[4.07061282 3.47508565 1.86212018]
Iteration 110: Best Cost = 0.11144876730195379 | sol:[4.0788724  3.46552456 1.85441703]
Iteration 120: Best Cost = 0.1042170436357890